# Improved Real-Time trailing Stop

This Final Project presents an improved real-time trailing-stop strategy. It keeps track in real time of max, min, mean, VOL (volatility), FD (fractal dimension) and the returns ($r_i$) per 6 minute interval. Additionally, the code keeps track of the balance of a currency pair at each hour and also the profit (or loss).

1. The code does this by calculating the volatility as a first order calculation (max – min). After that, 200 Keltner Channels are created in the next interval using the mean and the volatility from the previous interval:

    $Keltner Channel Upper Band = Mean Value + n0.025VOL, \ n \in range(1,100)$ 

    $Keltner Channel Lower Band = Mean Value - n0.025VOL, \ n \in range(1,100)$ 

    The number of crosses with the Keltner bands get counted during the 6 minute interval and then the fractal dimension is calculated as: $\frac{nb\_cross}{VOL}$.
   
   Extra info: this is based on brownian montion with the Mean Value as the best approximation for the next 6 minutes and the 0.025 is the step size. 


2. The returns are calculated using the following formula:
$r_i = \frac{(P_i - P_{i-1})}{P_{i-1}}$ with $P_i$ the mean price in the current interval and $P_{i-1}$ the mean price in the previous interval (see HWK 3 explanation).


3. A trade can have up to four layers. After each hour we check, if our stop loss is reached. If the position is doing well, we add 100 extra currency units to our trade until after the fourth layer then we stop buying or selling. The stop loss also decreases with each layer to protect our profits: 0.250% &rarr; 0.150% &rarr; 0.100% &rarr; 0.050%.

    This stop loss gets compared with an approximation of the returns, using the last 10 return values. For example: $r_{10} = \sum_{i=2}^{10} r_{i}$


4. The code also keeps tracks of the average buy or sell price to be able to calculate the exact profits when closing a trade.


5. Note that the first 6 minute checkpoint doesn't yield a FD or a return value and these are set to Null, because there is no previous information yet.


6. The code uses a regression model to predict the return of the next hour


7. The code also implements a sanity check to see if the provided data by Polygon makes sense and is correct and a check for when a certain pair has low liquidity (VOL=0)

## Best Practices

__Best practice 1:__ Goal is to build an optimized trailing-stop strategy to trade FOREX markets using ML techniques for prediction.

Spread througout the code: 

   -__Best practice 2__: Collecting all fields that are relevant
    
   -__Best practice 3__: Maintaining the consistency of field values
   
   -__Best practice 4__: Dealing with missing and wrong data

## Imports

In [ ]:
import os
import time
import pickle
import joblib
import datetime
import requests
import numpy as np
import pandas as pd
from math import floor
from threading import Thread
from dotenv import load_dotenv
from datetime import timedelta
from pymongo import MongoClient
from pycaret.regression import *
from keras.models import Sequential
from polygon.rest import RESTClient  # polygon.rest allows to just install the full polygon package without any problems
from keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from scipy.interpolate import InterpolatedUnivariateSpline

In [ ]:
# Importing the extra classes
import Portfolio
import PredictionModel

## Extra Classes

I chose to move the extra classes to different python files and import them to keep this notebook concise and understanable. For documentation and more information about the extra classes that I use you can open their python files.

## Helper functions

The functions below are helper functions used in the main function:

In [ ]:
"""
Function slightly modified from polygon sample code to format the date string 

Arguments:
    - ts: last_trade.timestamp (timestap from the last_trade object)
    
Returns:
    - datetime object
"""
def ts_to_datetime(ts) -> str:
    return datetime.datetime.fromtimestamp(ts / 1000.0).strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
"""
This function intializes a dictionary for the currency pair with all the parameters that we need to keep track off
during our trades.

This function also initializes a start mean and a start volatility to begin the code with, used in the sanity check. 
For the the start mean the current price is used and for the start volatility 2% is used, with the reasoning that 
it is statistically very unlikely to have such high volatility (especially in a 6 minute interval).

Arguments:
    - currency_pair: Tuple, with the currency pair
    
    - client: Object, Polygon REST api client

Returns:
    - Dictionary, with al the parameter to keep track off
"""
def initialize_dictionary(currency_pair, client, position):
    
    ticker = "C:" + currency_pair[0] + currency_pair[1]
    last_quote = client.get_last_quote(ticker)
    prev_mean = (last_quote.ask_price + last_quote.bid_price)/2
    portfolio = Portfolio.Portfolio(currency_pair[0], currency_pair[1], client)
    
    return {"prev_band_nb": 0, "nb_crosses": 0,"prev_vol": 0.02, "prev_mean": prev_mean,
            "max": 0, "min": 9999, "portfolio": portfolio, "position": position, "prev_pred": 0}

In [ ]:
"""
This the sanity check (best practice 4) for the returned values of Polygon. I noticed that Polygon sometimes returns an 
incorrect value that is almost exact the same as 1/correct. This makes me believe that Polygon probably sometimes switches 
the from and to currency.

I use the reasoning that it is statiscally very unlikely that avg_price > prev_mean + 750*0.025*prev_vol and,
avg_price < prev_mean - 750*0.025*prev_vol

Arguments:
    - client: Object of Polygon client
    
    - avg_price: Float, avg price of currency pair
    
    - currency_pair: Tuple, with the currency pair
    
    - curr_dict: Dictionary, contains all the required parameters

Returns:
    - avg_price

"""
def filter_out_incorrect_values(client, avg_price, currency_pair, curr_dict):
    
    if (avg_price > curr_dict["prev_mean"] + 750*0.025*curr_dict["prev_vol"] or avg_price < curr_dict["prev_mean"] - 750*0.025*curr_dict["prev_vol"]) and curr_dict["prev_vol"] != 0:
        print("Very unlikely value detected:", avg_price)
        
        # Call the API again with the required parameters, using different call method
        try:
            ticker = "C:" + currency_pair[0] + currency_pair[1]
            last_quote = client.get_last_quote(ticker)
            avg_price = (last_quote.ask_price + last_quote.bid_price)/2
            
        except:
            pass
    
    if (avg_price > curr_dict["prev_mean"] + 750*0.025*curr_dict["prev_vol"] or avg_price < curr_dict["prev_mean"] - 750*0.025*curr_dict["prev_vol"] ) and curr_dict["prev_vol"] != 0:
        # Value is still wrong, discard the value
        avg_prive = None  # None values are ignored when doing operation on NoSql database, such as AVG() + get removed in preprocessing
    
    return avg_price

In [ ]:
"""
The update_cross_counter() function is used to count the number of times that a given value "crosses" a specific threshold. 
This threshold is calculated using an average price and a previous volume and mean value.
"""
def update_cross_counter(avg_price, curr_dict):
    
    prev_vol = curr_dict["prev_vol"]
    prev_mean = curr_dict["prev_mean"]
    # Assumption point on a band is not a cross, has to be > or <

    # Keltner bands, count the number of crosses in realtime
    prev_band_nb = curr_dict["prev_band_nb"]

    # Formula used to calculate the band nb:
    # band_nb = floor((abs(avg_price - prev_mean))/(0.025*prev_vol))

    if avg_price > prev_mean + 0.025*prev_vol and prev_vol != 0:
        band_nb = floor((avg_price - prev_mean)/(0.025*prev_vol))

    elif avg_price < prev_mean - 0.025*prev_vol and prev_vol != 0:
        band_nb = floor((avg_price - prev_mean)/(-0.025*prev_vol))

    else:
        band_nb = 0  # lays within the keltner channel

    # Can't go over 100 for the band_nb
    band_nb = 100 if band_nb > 100 else band_nb


    # Set new prev_band_nb and increment crosses counter
    curr_dict["prev_band_nb"] = band_nb
    curr_dict["nb_crosses"] = curr_dict["nb_crosses"] + abs(band_nb - prev_band_nb)
    
    return curr_dict

In [ ]:
"""
This function is called every 6 minutes to aggregate the data, store it in the aggregate table, and then delete the raw data

Arguments:
    - engine: Engine object from sqlalchemy
    
    - curr: Tuple, with the currency pair
    
    - curr_dict: Dictionary, contains all the required parameters
    
    - start_time: Float, the start time in seconds of the code

Returns:
    - None

"""
def aggregate_raw_data_tables(mongo_client, curr, curr_dict, start_time, key_alpha):


    # Select the database that contains the collections
    db = mongo_client["FOREX_currencypairs3"]
    
    # Use the .aggregate() method to run the aggregation pipeline and return the result
    result = db[curr[0]+curr[1]+curr_dict["position"]+"_raw"].aggregate([
    {
        "$group": {
            "_id": None,
            "average_price": { "$avg": "$fxrate" },
            "maxtick": { "$max": "$ticktime" }
            }
        }
    ])
    
    # Result is a cursor object, what is a form of an iterable
    grouped_document = result.next()
    mean = grouped_document["average_price"]
            
    # vol = (max - min)/mean
    vol = (curr_dict["max"] - curr_dict["min"])/mean  # noramlizing using the mean price
    # previous vol
    curr_dict["prev_vol"] = vol

    # Calculate the returns here
    if time.time() < start_time + 390:
        return_ = None
    else:
        return_ = (mean - curr_dict["prev_mean"])/curr_dict["prev_mean"]

    # previous mean
    curr_dict["prev_mean"] = mean

    # Prevent division by 0 error and set first FD to null, do a sanity check for low liquidity, best practice 4
    if time.time() < start_time + 390 or vol == 0:
        frac_dem = None
    else:
        frac_dem = curr_dict["nb_crosses"]/vol  # normalizing using vol
        
    # Create the documents to insert
    document_agg = {"inserttime": grouped_document["maxtick"], "avgfxrate": mean}
    document_maxmin = {"inserttime": grouped_document["maxtick"], "max": curr_dict["max"], "min": curr_dict["min"], 
                       "VOL": vol, "mean": mean, "FD": frac_dem, "return": return_}

    # Insert the documents into the collections
    db[curr[0]+curr[1]+curr_dict["position"]+"_agg"].insert_one(document_agg)
    
    if curr[0]+curr[1] != "EURUSD":
        db[curr[0]+curr[1]+curr_dict["position"]+"_maxmin"].insert_one(document_maxmin)
    
    # Add the two extra features to the EUR/USD pair
    if curr[0]+curr[1] == "EURUSD":
        
        endpoint = 'https://www.alphavantage.co/query'
        params = {
          'function': 'CURRENCY_EXCHANGE_RATE',
          'from_currency': 'XAU',
          'to_currency': 'USD',
          'apikey': key_alpha
        }

        try:
            response_gold = requests.get(endpoint, params=params)
            response_oil = requests.get(f"{endpoint}?function=GLOBAL_QUOTE&symbol=OIL&apikey={key_alpha}")
        except:
            pass
        
        # Improvement: implement a sanity check for this data as well 

        # Parse the JSON responses
        data_oil = response_oil.json()
        data_gold = response_gold.json()

        # Get the current oil_price and gold price
        oil_price = data_oil["Global Quote"]["05. price"]
        gold_price = data_gold['Realtime Currency Exchange Rate']['5. Exchange Rate']

        # Convert the gold price to a float
        gold_price = float(gold_price)
        
        document_maxmin = {"inserttime": grouped_document["maxtick"], "max": curr_dict["max"], "min": curr_dict["min"], "VOL": vol, 
                           "mean": mean, "FD": frac_dem, "gold_price": gold_price, "oil_price": oil_price, "return": return_}
        
        db[curr[0]+curr[1]+curr_dict["position"]+"_maxmin"].insert_one(document_maxmin)

In [ ]:
"""
Function which clears the raw data tables once we have aggregated the data in a 6 minute interval and resets the number of
crosses, band_nb, max and min parameters for the Keltner Channels.

Arguments:
    - mongo_client: The MongoDB client of our NoSQL database
    
    - curr: Tuple, with the currency pair
    
    - curr_dict: Dictionary, contains all the required parameters
    
Returns:
    - None
"""
def reset_raw_collection(mongo_client, curr, curr_dict):
        
    # Select the database that contains the collection you want to delete
    db = mongo_client["FOREX_currencypairs3"]

    # Delete the collection
    db.drop_collection(curr[0]+curr[1]+curr_dict["position"]+"_raw")

    # reset number of crosses, band_nb and max, min
    # Assumption: we start each time in band nb 0, not 100% correct, but should not lead to big differences
    curr_dict["nb_crosses"] = 0
    curr_dict["band_nb"] = 0
    curr_dict["max"] = 0
    curr_dict["min"] = 9999  # min is set to a random high number


In [ ]:
"""
This code defines a function determine_strategy_next_hour that takes in a mongo_client, curr, and curr_dict as input. 
The mongo_client is a MongoClient object that is used to connect to a MongoDB database. The curr input is a tuple containing 
the current currency pair being traded, and the curr_dict input is a dictionary containing information about the current trade 
position and portfolio.

Arguments:
    - mongo_client: The MongoDB client of our NoSQL database
    
    - curr: Tuple, with the currency pair
    
    - curr_dict: Dictionary, contains all the required parameters
    
Returns:
    - Reinvest_flag, determines wether we reinvest or stay neutral on our trade position
"""
def determine_strategy_next_hour(mongo_client, curr, curr_dict):
    
    # Set reinvestflag to true
    reinvest_flag = True
    
    # Get the database
    db = mongo_client["FOREX_currencypairs3"]
    
    # Get the needed collections for the prediction and comparision
    collection_maxmin = db[curr[0]+curr[1]+curr_dict["position"]+"_maxmin"]
    
    # Load the LSTM model
    predObj = PredictionModel.Predictionmodel()
    yhat = predObj.predict_with_LSTM(mongo_client, curr, curr_dict["position"])
    
    # Load the trained regression model
    # loaded_model = load_model('pipeline_' + curr[0] + curr[1] + curr_dict["position"])

    # Get the exact hour return from the previous hour
    result = collection_maxmin.find({},{"return": 1, "_id":0}).sort([("_id", -1)]).skip(10).limit(10)
    R10_prev_hour = sum(value["return"] for value in result)
           
    # Make prediction using the PredictionModel class
    # predObj = PredictionModel.PredictionModel()
    # predObj.best_model = loaded_model
    # result = predObj.predict_return_next_hour(mongo_client, curr, curr_dict["position"], 0, 0)

    # next_hour_pred = result.sum(axis=0)
    # next_hour_pred = next_hour_pred['Label']/100000
    
    next_hour_pred = yhat
    
    # Calculate the previous error
    prev_error = curr_dict["prev_pred"] - R10_prev_hour
    
    # Update the previous prediction
    curr_dict["prev_pred"] = next_hour_pred
    
    # Correct with the error
    corrected_pred = abs(next_hour_pred + prev_error)
    
    # Implement the 4 options discussed to 
    if abs(next_hour_pred) >= 0 and corrected_pred >= 0:
        reinvest_flag = True
    
    elif (abs(next_hour_pred) < 0 and corrected_pred > 0) or (abs(next_hour_pred) > 0 and corrected_pred < 0):
        reinvest_flag = False
        
    else:
        realised_loss = curr_dict["portfolio"].close_trade(curr_dict["position"])
        print(f"[UPDATE], realized loss of {curr[0]}_{curr[1]} is {realised_loss}")

        # Stop the thread and stop the trade
        raise ValueError(f"Stop loss limit reached in window {window}")
        
    return reinvest_flag

In [ ]:
"""
This function reviews the position every hour to check if we should close the trade, stay neutral or reinvest.
"""
def review_position(mongo_client, curr, curr_dict):
        
    # Define the stop loss limits
    stop_loss_limits = [0.00250, 0.00150,  0.00100, 0.00050]
    
    # Select the database that contains the collection we want to count
    db = mongo_client["FOREX_currencypairs3"]

    # Select the collection we want to count
    # Use the .count_documents() method to count the number of documents in the collection
    collection = db[curr[0]+curr[1]+curr_dict["position"]+"_maxmin"]
    nb_documents = collection.count_documents({})
    
    window = min(int(nb_documents/10), 4)
    
    # Use the Portfolio class to calculate the exact return at this moment
    exact_return = curr_dict["portfolio"].exact_return(curr_dict["position"])
    
    # Second option, use an estimated return, based on the previous hour
    # result = collection_maxmin.find({},{"return": 1, "_id":0}).sort([("_id", -1)]).skip(10).limit(10)
    # estimated_return = sum(value["value"] for value in result)

    if abs(exact_return) > stop_loss_limits[window-1]:
        print(f"[UPDATE], stop loss limit is reached for {curr[0]}_{curr[1]} at {stop_loss_limits[window-1]} and trade is closed")

        realised_loss = curr_dict["portfolio"].close_trade(curr_dict["position"])
        print(f"[UPDATE], realized loss of {curr[0]}_{curr[1]} is {realised_loss}")

        # Stop the thread and stop the trade
        raise ValueError(f"Stop loss limit reached in window {window}")
    
    reinvest_flag = determine_strategy_next_hour(mongo_client, curr, curr_dict)

    if curr_dict["position"] == "LONG" and reinvest_flag:
        curr_dict["portfolio"].buy_curr(100)

    elif curr_dict["position"] == "SHORT" and reinvest_flag:
        curr_dict["portfolio"].sell_curr(100)
        
    collection_balance = db[curr[0]+curr[1]+curr_dict["position"]+"_balance"]
    
    profit = curr_dict["portfolio"].calculate_profit(curr_dict["position"])
    balance = curr_dict["portfolio"].convert_balance(curr_dict["position"])
    exact_return = curr_dict["portfolio"].exact_return(curr_dict["position"])
    
    # Create the document to insert
    document = {"balance_in_USD": balance, "profit_loss_in_USD": profit, "exact_return": exact_return}

    # Insert the document into the collection
    collection_balance.insert_one(document)
                

## Mainfunction

Each currency pair runs on a different thread to exploit parallelism and each thread will call this function.

In [ ]:
"""
This main function repeatedly calls the polygon api every 1 seconds for 10 hours and stores the results.

Arguments:
    - currency_pairs: Tuple with the base and quote currency
    
    - duration, Integer, time in seconds for how long the trade should last
    
    - mongo_client: The MongoDB client of our NoSQL database
    
    - position_type: String, that is either "SHORT" or "LONG" depending on the type of the position
    
Returns:
    - None
"""
def mainThread(currency_pair, mongo_client, duration, position_type):
    
    # API key to access Polygon
    key_poly = os.environ.get("api-key")
    # API key to access alphavantage
    key_alpha = os.environ.get("api-key2")
    
    # Open a RESTClient for making the api calls
    client = RESTClient(key_poly)
    
    # Create dictionary for the currency_pair with the parameters to keep track of
    curr_dict = initialize_dictionary(currency_pair, client, position_type)
        
    if position_type == "LONG":
        curr_dict["portfolio"].buy_curr(100)

    else:
        curr_dict["portfolio"].sell_curr(100)
    
    # Start the trading loop for the currency pair
    start_trade_loop(client, mongo_client, currency_pair, curr_dict, position_type, duration, key_alpha)
    
    # After the while loop the trade gets closed and profit/loss is realised by closing the trade
    realised_profit = curr_dict["portfolio"].close_trade(position_type)
    print(f"[UPDATE], Closed trade of {currency_pair[0]}_{currency_pair[1]} with realized profit/loss of {realised_profit}")

In [ ]:
"""
The loop for the trading process, keeps running for the duration passed by the user or gets closed early if a stop loss is reached.

Arguments:
    - mongo_client: The MongoDB client of our NoSQL database
    
    - currency_pair: Tuple, with the currency pair
    
    - curr_dict: Dictionary, contains all the required parameters
    
    - currency_pairs: Tuple with the base and quote currency
    
    - position_type: String, that is either "SHORT" or "LONG" depending on the type of the position
    
    - duration, Integer, time in seconds for how long the trade should last
    
Returns:
    - None
"""
def start_trade_loop(client, mongo_client, currency_pair, curr_dict, position_type, duration, key_alpha):
    
    # Create run intervals using the user specified duration
    start_6min_interval = start_1hour_interval = start_time = time.time()
    end_time = start_time + duration
    
    print(f"[UPDATE] Currency pair {currency_pair[0]}_{currency_pair[1]} started at {time.localtime(start_time).tm_hour}h {time.localtime(start_time).tm_min}min.")

    
    # Loop that runs until the total duration of the trade until the end time is reached
    while time.time() < end_time:
        
        # Aggregate the data and clear the raw data tables
        if time.time() > start_6min_interval + 360:
            print(f"[UPDATE] Currency pair {currency_pair[0]}_{currency_pair[1]} aggregated at {time.localtime(start_6min_interval + 360).tm_hour}h {time.localtime(start_6min_interval + 360).tm_min}min.")
            aggregate_raw_data_tables(mongo_client, currency_pair, curr_dict, start_time, key_alpha)
            reset_raw_collection(mongo_client, currency_pair, curr_dict)
            
            # Start new 6 minute interval
            start_6min_interval = time.time()

        if time.time() > start_1hour_interval + 3610:
            review_position(mongo_client, currency_pair, curr_dict)
            
            # Start new 1 hour interval
            start_1hour_interval = time.time()
        

        # Set the input variables to the API
        from_ = currency_pair[0]
        to = currency_pair[1]

        # Call the API with the required parameters
        try:
            resp = client.get_real_time_currency_conversion(from_, to, amount=100, precision=2)
        except:
            continue
            
        # This gets the Last Trade object defined in the API Resource
        last_trade = resp.last    

        # Format the timestamp from the result
        dt = ts_to_datetime(last_trade.timestamp)
        
        # Calculate the price by taking the average of the bid and ask prices
        avg_price = (last_trade.bid + last_trade.ask)/2

        # set needed parameters
        curr_max = curr_dict["max"]
        curr_min = curr_dict["min"]
        
        # Do a sanity check on the retrieved value from Polygon, best practice 4
        avg_price = filter_out_incorrect_values(client, avg_price, currency_pair, curr_dict)

        if last_trade.ask > curr_max:
            # Update current maximum
            curr_dict["max"] = last_trade.ask

        elif last_trade.bid < curr_min:
            # Update current minimum
            curr_dict["min"] = last_trade.bid 
        
        update_cross_counter(avg_price, curr_dict)

        # Get a reference to the NoSQL database and the collection, on the first reference the database and 
        # collections get created, on first call the database and collection get created automatically 
        db = mongo_client["FOREX_currencypairs3"]
        collection = db[currency_pair[0]+currency_pair[1]+curr_dict["position"]+"_raw"]

        # Create the document to insert, best practice 2
        document = {"ticktime": dt, "fxrate": avg_price}

        # Insert the document into the collection
        collection.insert_one(document)

        # Time the code so we have approx 1 data point per second
        time.sleep(0.980)

## Training the ML Models

In [ ]:
curr_list = [ ("EUR", "USD"), ("GBP", "USD"), ("USD","CAD"), ("USD", "CHF"),  ("USD", "HKD"), ("AUD","USD"), ("NZD","USD"), ("USD","SGD")]

# load environment variables, used to hide the mongo cluster access password and the API key
# the .env file would not be shared to ensure the confidentially
load_dotenv()

# configure the MongoDB database
mongo_cluster = os.environ.get("cluster_access")
mongo_client = MongoClient(mongo_cluster)

for curr in curr_list:

    #Before starting the trading code, train the regression models and the LSMT neural net using the training data from the previous days
    
    # Prediction Objects for LSMT models
    predObj3 = PredictionModel.PredictionModel()
    predObj4 = PredictionModel.PredictionModel()
    
    # Data preprocessing, best practices 3 and 4
    dataframes_long = predObj3.data_preprocessing2(mongo_client, curr, "LONG")
    dataframes_short = predObj4.data_preprocessing2(mongo_client, curr, "SHORT")

    # sorted_datasets = predObj.data_preprocessing(mongo_client, curr, "SHORT")
    
    predObj3.train_LSMT_model(curr, dataframes_long[0], dataframes_long[1], dataframes_long[2], "LONG")
    predObj3.train_LSMT_model(curr, dataframes_short[0], dataframes_short[1], dataframes_short[2], "SHORT")
    
#     result1 = predObj1.train_model(mongo_client, curr, "LONG")
#     result2 = predObj2.train_model(mongo_client, curr, "SHORT")
#     predObj1.best_model = result1
#     predObj2.best_model = result2

#     sorted_datasets = predObj2.data_preprocessing(mongo_client, curr, position_type)
    
    
    # Saving the model, best practice 16
#     save_model(predObj1.best_model, 'pipeline_' + curr[0] + curr[1] + "_Regression_SHORT")
#     save_model(predObj2.best_model, 'pipeline_' + curr[0] + curr[1] + "_Regression_LONG")
    
    # print(f"[UPDATE] The best model of {curr[0]}_{curr[1]} has an r2_score of {r2_score} with sorting option {predObj.sort_option}")

## Start the Main Function

In [ ]:
# Drop the database if it already exists to prevent errors.CollectionInvalid
# Delete the database, make sure to have admin acces for this
mongo_client.drop_database("FOREX_currencypairs3")

# create threads for BUY positions
buy_list = [ ("EUR", "USD"), ("GBP", "USD"), ("USD","CAD"), ("USD", "CHF"),  ("USD", "HKD"), ("AUD","USD"), ("NZD","USD"), ("USD","SGD")]
threads_buy = [Thread(target=mainThread, args=(pair, mongo_client, 36120, "LONG")) for pair in buy_list]

# create threads for SHORT positions
sell_list = [ ("EUR", "USD"), ("GBP", "USD"), ("USD","CAD"), ("USD", "CHF"),  ("USD", "HKD"), ("AUD","USD"), ("NZD","USD"), ("USD","SGD")]
threads_sell = [Thread(target=mainThread, args=(pair, mongo_client, 36120, "SHORT")) for pair in sell_list]

threads = threads_buy + threads_sell

# Mention advantages of Threads:
# - Fixes timing issues in the code (best practice 1)
# - Allows for separate failures (best practice 4)

# start the threads
for thread in threads:
    thread.start()
    
# wait for the threads to complete
for thread in threads:
    thread.join()

## Useful scripts

### Transfer SQL database into a MongoDB database

In [ ]:
def sqlite_to_mongodb(engine, mongodb_uri, mongodb_database, mongodb_collection1, mongodb_collection2):
    
    with engine.begin() as conn:

        # Connect to the MongoDB database
        client = pymongo.MongoClient(mongodb_uri)
        db = client[mongodb_database]
        collection1 = db[mongodb_collection1]
        collection2 = db[mongodb_collection2]

        # Iterate over the rows in the SQLite database
        result = conn.execute(text('SELECT * FROM' + ' ' + mongodb_collection1))
        for row in result:
            # Insert the row into the MongoDB collection as a document
            collection1.insert_one(dict(row))
        
        result = conn.execute(text('SELECT * FROM' + ' ' + mongodb_collection2))
        for row in result:
            # Insert the row into the MongoDB collection as a document
            collection2.insert_one(dict(row))

### Cubic Interpolation of Financial Data

In [ ]:
def cubic_spline_interpolation_daydata(data, handle_missing = 'interpolate', extrapolate = 0):
    # Extract the dates and values from the input data
    dates = [dat[0] for dat in data]
    values = [dat[1] for dat in data]
    
    # Best practice 4:
    # Handle missing data
    if handle_missing == 'interpolate':
        # Interpolate missing values
        values = [value if value is not None else np.nan for value in values]
    elif handle_missing == 'drop':
        # Drop missing values
        values = [value for value, date in zip(values, dates) if value is not None]
        dates = [date for value, date in zip(values, dates) if value is not None]
    elif handle_missing == 'fill':
        # Fill missing values with the previous value
        prev_value = None
        for i, (value, date) in enumerate(zip(values, dates)):
            if value is None:
                values[i] = prev_value
            else:
                prev_value = value
    
    # Create the interpolation function
    interpolation_function = InterpolatedUnivariateSpline(np.arange(0,24*(len(dates))/2,12), values, k=3)

    # Interpolate the values for the output dates
    output_values = interpolation_function(np.arange(0,24*len(dates)/2,), ext = extrapolate)

    # Return the interpolated values
    return output_values

In [ ]:
def cubic_spline_interpolation_hourlydata(values, handle_missing = 'interpolate', extrapolate = 0):
    
    # Data already checked in daydata function, use output of that function

    # Create the interpolation function
    interpolation_function = InterpolatedUnivariateSpline(np.arange(0,len(values)*60, 60), values, k=3)


    # Interpolate the values for the output dates
    output_values = interpolation_function(np.arange(0,len(values)*60 + 60, 6), ext = extrapolate)

    # Return the interpolated values
    return output_values

## Further Improvements

The first improvement would be to use the techniques dicussed in class to get a better model for the
predictions.


The second improvement would be to extend the conversion method in the Portfolio class to also allow
for cross pairs to be traded.


There is also still a bit of time lost when aggregating and resetting the raw data tables. After the code
runs for 10 hours this results in a small delay of ≈ 45s. When running the code non stop this could
eventually lead to timing issues. The quick fix in HWK 3 was to give the code a 2 minute buffer at the
end to finish to full 10 hour run. A better fix for this would be done by calling a separate process (thread)
to aggregate and reset the raw data tables. This would require the code to work with 2 sets of tables:
one set that is being activily updated, while the other set can be aggregated and reset.


Potentially, there could also be a documentation document generated to make the code more understandable.


Maybe add a visual display that keeps live track of the trades and gives feedback


Do a stastical analysis of the models to see where I can still improve